In [1]:
import pandas as pd

In [23]:
california_houses = pd.read_csv("datasets/California_Houses.csv", index_col=0)
california_houses.head(10)

,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
Median_House_Value,,,,,,,,,,,,,
452600.0,8.3252,41,880,129,322,126,37.88,-122.23,9263.040773,556529.158342,735501.806984,67432.517001,21250.213767
358500.0,8.3014,21,7099,1106,2401,1138,37.86,-122.22,10225.733072,554279.850069,733236.884360,65049.908574,20880.600400
352100.0,7.2574,52,1467,190,496,177,37.85,-122.24,8259.085109,554610.717069,733525.682937,64867.289833,18811.487450
341300.0,5.6431,52,1274,235,558,219,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
342200.0,3.8462,52,1627,280,565,259,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
269700.0,4.0368,52,919,213,413,193,37.85,-122.25,7768.086571,555194.266086,734095.290744,65287.138412,18031.047568
299200.0,3.6591,52,2535,489,1094,514,37.84,-122.25,6843.020847,554364.406936,733249.808578,64315.991742,17538.209972
241400.0,3.1200,52,3104,687,1157,647,37.84,-122.25,6843.020847,554364.406936,733249.808578,64315.991742,17538.209972
226700.0,2.0804,42,2555,665,1206,595,37.84,-122.26,6363.948013,554949.749519,733820.824419,64751.463863,16744.406584


In [24]:
# shuffle data
california_houses = california_houses.sample(frac=1).reset_index(drop=True)
california_houses


,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
0,4.0417,25,1729,383,769,352,37.53,-122.33,7446.186715,534099.472382,712176.586660,44232.140669,28556.628458
1,2.9315,46,2126,409,1292,414,33.98,-118.34,14388.033086,11971.703102,178193.113339,492109.797605,560096.721390
2,5.3787,42,1761,329,480,255,32.85,-117.26,2630.953643,161861.190999,17561.136402,652831.219931,720807.729742
3,2.4196,46,2038,534,1250,525,34.08,-118.32,18514.457064,7677.854885,185972.904929,484803.201924,552823.125014
4,1.9667,26,1485,334,406,180,38.94,-119.97,167281.674700,564942.946799,736867.636260,245176.155791,250722.291184
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,2.6321,21,4124,1054,2162,998,37.70,-122.13,8541.594694,535731.733280,714565.084262,45603.835647,27742.281169
20636,6.9903,14,2351,341,1128,363,36.83,-119.83,174890.203495,340652.210477,518316.249701,191438.000614,252884.004926
20637,6.0100,43,1061,208,514,208,34.23,-118.24,34050.024939,19769.472037,195868.184379,477228.557881,545259.423859
20638,1.6797,38,1838,388,1203,373,35.36,-119.01,118010.989399,161407.990191,339800.450141,338968.126894,406648.126540


In [ ]:
# 1.splitting data

total_rows = california_houses.shape[0]
train_set = california_houses.iloc[:(int)(total_rows * 0.7)]
validation_set = california_houses.iloc[(int)(total_rows * 0.7):(int)(0.85 * total_rows)]
test_set = california_houses.iloc[(int)(0.85 * total_rows):]


# display(train_set.head())
# display(validation_set.head())
# display(test_set.head())

,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
0,4.0417,25,1729,383,769,352,37.53,-122.33,7446.186715,534099.472382,712176.586660,44232.140669,28556.628458
1,2.9315,46,2126,409,1292,414,33.98,-118.34,14388.033086,11971.703102,178193.113339,492109.797605,560096.721390
2,5.3787,42,1761,329,480,255,32.85,-117.26,2630.953643,161861.190999,17561.136402,652831.219931,720807.729742
3,2.4196,46,2038,534,1250,525,34.08,-118.32,18514.457064,7677.854885,185972.904929,484803.201924,552823.125014
4,1.9667,26,1485,334,406,180,38.94,-119.97,167281.674700,564942.946799,736867.636260,245176.155791,250722.291184


,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
14447,3.3320,16,4199,794,2140,722,37.02,-119.57,199268.614273,351133.407030,526603.129018,208773.878360,266289.731775
14448,3.1250,52,1350,266,490,257,38.06,-122.23,2357.695195,571714.565229,750901.624427,85844.871564,36377.984136
14449,7.5794,37,1795,285,791,261,37.35,-122.09,14366.701676,504968.336707,683099.413379,17487.471819,55924.018200
14450,5.8625,8,2683,416,1154,399,35.64,-117.70,194853.936777,183389.255123,328918.863861,419533.189179,483845.384438
14451,4.5293,19,4187,684,1827,605,38.34,-122.31,22819.013249,600232.396535,779568.770517,117545.904641,63829.128504


,Median_Income,Median_Age,Tot_Rooms,Tot_Bedrooms,Population,Households,Latitude,Longitude,Distance_to_coast,Distance_to_LA,Distance_to_SanDiego,Distance_to_SanJose,Distance_to_SanFrancisco
17544,4.8382,45,989,174,504,180,37.51,-122.25,6283.232235,527560.387633,705761.789061,37017.709634,33412.214350
17545,4.1518,22,2101,399,1551,371,32.58,-117.05,8769.246350,197738.061745,18329.679021,688546.543564,756508.107401
17546,9.3586,32,1433,199,498,201,37.36,-122.10,13406.692879,506380.751081,684516.381035,18497.626630,54511.657761
17547,1.3867,30,1417,589,1615,540,34.05,-118.29,20250.184009,4274.381785,181651.821483,489134.296694,557153.987477
17548,3.5580,42,1768,372,1155,368,33.90,-118.08,18246.411660,22679.440550,156946.406041,514414.176507,582442.323092
